## HashMap
---

In [2]:
:dep xxhash-rust = { features = ["xxh3"] }

In [3]:
use std::fmt::Debug;

In [4]:
#[derive(Debug)]
struct HashMap<V: Debug> {
    bins: Vec<Vec<(Vec<u8>, V)>>,
    size: usize,
}

In [5]:
impl<V: Debug> HashMap<V> {
    fn new() -> Self {
        let mut hm = Self {
            bins: Vec::<Vec<(Vec<u8>,V)>>::with_capacity(0),
            size: 0,
        };
        hm.bins.push(Vec::<(Vec<u8>,V)>::with_capacity(0));
        hm
    }
    fn add_bins(&mut self) {
        for _ in 0..std::cmp::max(1, self.num_bins()) {
            self.bins.push(Vec::<(Vec<u8>,V)>::with_capacity(0));
        }
        self.rehash();
        self.rehash();
    }
    fn num_bins(&self) -> usize {
        self.bins.len()
    }
    fn load_factor(&self) -> f64 {
        (self.size as f64)/(self.bins.len() as f64)
    }
    fn hash(&self, key: &Vec<u8>) -> usize {
        use xxhash_rust::xxh3::xxh3_64;
        ((self.num_bins() as f64) * (xxh3_64(key).min(u64::MAX - 1) as f64)/
         (u64::MAX as f64)
        ).floor() as usize
    }
    fn rehash(&mut self) {
        for n in 0..self.bins.len() {
            let M = self.bins[n].len();
            for m in 0..M {
                if m >= self.bins[n].len() {
                    break;
                }
                let hv = self.hash(&self.bins[n][m].0);
                if hv != n {
                    let kv = self.bins[n].swap_remove(m);
                    self.bins[hv].push(kv);
                }
            }
        }
    }
    fn set(&mut self, key: Vec<u8>, val: V) {
        let hv = self.hash(&key);
        let indx = self.bins[hv].iter().position(|x|x.0==key);
        if indx.is_none() {
            self.bins[hv].push((key, val));
            self.size += 1;
        } else {
            self.bins[hv][indx.unwrap()] = (key, val);
        }
        if self.load_factor() > 8.0 {
            self.add_bins();
        }
    }
    fn get(&mut self, key: &Vec<u8>) -> Option<&V> {
        let hv = self.hash(key);
        let indx = self.bins[hv].iter().position(|x|x.0==*key);
        if indx.is_none() {
            None
        } else {
            Some(&self.bins[hv][indx.unwrap()].1)
        }
    }
}

In [6]:
let mut hm = HashMap::<u64>::new();
println!("num_bins = {}", hm.num_bins());
hm

num_bins = 1


HashMap { bins: [[]], size: 0 }

In [7]:
hm.set(vec![1,1,1], 111);
hm.set(vec![2,2,2], 222);
hm.set(vec![3,3,3], 333);
hm.set(vec![4,4,4], 444);
hm.set(vec![5,5,5], 555);
hm.set(vec![6,6,6], 666);
println!("num_bins = {}", hm.num_bins());
hm

num_bins = 1


HashMap { bins: [[([1, 1, 1], 111), ([2, 2, 2], 222), ([3, 3, 3], 333), ([4, 4, 4], 444), ([5, 5, 5], 555), ([6, 6, 6], 666)]], size: 6 }

In [8]:
hm.set(vec![7,7,7], 777);
hm.set(vec![8,8,8], 888);
hm.set(vec![9,9,9], 999);
hm.set(vec![10,10,10], 101010);
hm.set(vec![11,11,11], 111111);
hm.set(vec![12,12,12], 121212);
hm.set(vec![13,13,13], 131313);
println!("num_bins = {}", hm.num_bins());
hm

num_bins = 2


HashMap { bins: [[([1, 1, 1], 111), ([6, 6, 6], 666), ([5, 5, 5], 555), ([13, 13, 13], 131313)], [([2, 2, 2], 222), ([3, 3, 3], 333), ([4, 4, 4], 444), ([9, 9, 9], 999), ([8, 8, 8], 888), ([7, 7, 7], 777), ([10, 10, 10], 101010), ([11, 11, 11], 111111), ([12, 12, 12], 121212)]], size: 13 }

In [9]:
for n in 1..20 {
    println!("{:?} = {:?}", n, hm.get(&vec![n,n,n]));
};
println!("num_bins = {}", hm.num_bins());

1 = Some(111)
2 = Some(222)
3 = Some(333)
4 = Some(444)
5 = Some(555)
6 = Some(666)
7 = Some(777)
8 = Some(888)
9 = Some(999)
10 = Some(101010)
11 = Some(111111)
12 = Some(121212)
13 = Some(131313)
14 = None
15 = None
16 = None
17 = None
18 = None
19 = None
num_bins = 2
